# Data Streaming Algorithms

## HW2

See preprocessed notebook at https://github.com/salasin/DataStreamingAlgorithms

## Dry Part

$M$ is a data stream and $|M| = n$.

$A$ is $M$'s heavy hitters sketch and $|A| = k$.

(a) For every $j ∈ A, \tilde{f_j} \ge f_j$

The proof is by induction on the number of appearances of an element in the stream.

**Base case:**

After the 1st appearance of $j$ we have $f_j = 1$.

If $|A| < k$ before $j$'s appearance then it's added to the sketch and we get $\tilde{f_j} = 1 \ge f_j$.

Otherwise, if $|A| = k$ then we get $\tilde{f_j} = 1 + \tilde{f_{}}_{min} \ge 1 = f_j$ because $\tilde{f_{}}_{min} \ge 0$ because the counters are non-negative.

**Induction step:**

We denote by $\tilde{f_j}^{(i)}$ the value of $j$'s counter after $i$ appearances of $j$ and by $f_j^{(i)}$ the true count of $j$'s appearances (which is $i$).

We assume that $\tilde{f_j}^{(i)} \ge f_j^{(i)}$ holds for a given $i \ge 1$.

Then for $i + 1$ we get:

If $j \in A$ then we incremenet $\tilde{f_j}^{(i)}$ and we get $\tilde{f_j}^{(i + 1)} = \tilde{f_j}^{(i)} + 1 \ge f_j^{(i)} + 1 = f_j^{(i + 1)}$.

If $j \notin A$ and since $i \ge 1$ (not $j$'s first appearance) we can infer that $j$ was in $A$ and was removed. Therefore it must hold that $|A| = k$. At the time of $j$'s removal from $A$ we had $\tilde{f_j}^{(i)} = \tilde{f_{}}_{min}$. Now we want to re-insert $j$ to $A$ and we have a new $\tilde{f_{}}_{min}^{(new)}$ that is bigger than $\tilde{f_{}}_{min}$ because all counters are monotonically increasing. Then we get $\tilde{f_j}^{(i + 1)} = \tilde{f_{}}_{min}^{(new)} + 1 > \tilde{f_{}}_{min} + 1 = \tilde{f_j}^{(i)} + 1 \ge f_j^{(i)} + 1 = f_j^{(i + 1)}$.

(b) For every $j \in [M], \tilde{f_j} ≤ f_j + \tilde{f_{}}_{min}$

If $j \notin A$ we have $\tilde{f_j} = 0$ and the inequality holds since the right side is non-negative.

If $j \in A$, then $j$ was inserted to $A$ at some point and additional instances of $j$ may have been observed since. We denote with $\tilde{f_j}^{(curr)}$ the current value of the counter corresponding to $j$. We denote with $\tilde{f_{}}_{min}^{(curr)}$ the value of the counter that currently has the minimal value and with $\tilde{f_{}}_{min}^{(old)}$ the value of the counter that had the minimal value at the time $j$ was inserted. We denote with $f_{j}^{(before)}$ the count of $j$ instances before $j$'s insertion and with $f_{j}^{(after)}$ the similar count after the insertion.

We observe that $f_j = f_{j}^{(before)} + f_{j}^{(after)}$ and therefore $f_j \ge f_{j}^{(after)}$. We also observe that $\tilde{f_{}}_{min}^{(old)} \le \tilde{f_{}}_{min}^{(curr)}$ because all counters are monotonically increasing.

Therefore:

$\tilde{f_j} = \tilde{f_j}^{(curr)} = f_{j}^{(after)} + \tilde{f_{}}_{min}^{(old)} \le f_j + \tilde{f_{}}_{min}^{(curr)} = f_j + \tilde{f_{}}_{min}$

(c) $ֿ\sum_{j \in A} \tilde{f_j} = n$

Proof by induction on the size of the stream.

**Base case:**

When n = 1, i.e. when we process the first element $i$ in the stream, we add a new counter $\tilde{f_i} = 1$ to the sketch. Therefore:

$ֿ\sum_{j \in A} \tilde{f_j} = \tilde{f_i} = 1 = n$

**Induction step:**

We denote by $\sum_{j \in A}^{(n)} \tilde{f_j}$ the sum of all counters and by $\tilde{f_{j}}^{(n)}$ the value of $j$'s counter after $n$ elements from the stream were processed.

We assume that $ֿ\sum_{j \in A}^{(n)} \tilde{f_j} = n$ holds for a given $n$.

Then for $n + 1$ we get:

We start processing the $n + 1$ element $i$ so we incremenet $\tilde{f_i}$ (Note that it doesn't matter in this context whether $i$ was in $A$. If it wasn't we simply assume that before the incrementation $\tilde{f_i} = \tilde{f}_{min}$) and we get:

 $\sum_{j \in A}^{(n + 1)} \tilde{f_j} = ֿ\sum_{j \in A, j \neq i}^{(n + 1)} \tilde{f_j} + \tilde{f_i}^{(n + 1)} = \sum_{j \in A, j \neq i}^{(n + 1)} \tilde{f_j} + \tilde{f_i}^{(n)} + 1 = \sum_{j \in A}^{(n)} \tilde{f_j} + 1 = n + 1$

(d) $\tilde{f}_{min} \le \lfloor n/k \rfloor$ and hence $f_j \le \tilde{f_j} \le f_j + \lfloor n/k \rfloor$ for every $j \in A$.

From (c) we know that $ֿ\sum_{j \in A} \tilde{f_j} = n$. Also, the number of counters is $k$. Therefore, the average value of a counter is $\lfloor n / k \rfloor$. Since $\tilde{f}_{min}$ is the smallest of all counters then it must be below the average and therefore $\tilde{f}_{min} \le \lfloor n/k \rfloor$.

Combining this conclusion with the conclusions from (a) and (b) we can deduce that $f_j \le \tilde{f_j} \le f_j + \lfloor n/k \rfloor$ for every $j \in A$.

## Wet Part

In [40]:
import os.path
import pickle
import pandas as pd
import numpy as np
import sys
import time

In [41]:
data = pd.read_csv('2017-07-03.csv')
source_ips = data.loc[:, 'ipv4Src']
unique_source_ips = len(source_ips.unique())
print(f'numer of unique source ips: {unique_source_ips}')

numer of unique source ips: 9640


In [42]:
true_frequencies = {}
for source_ip in source_ips:
    true_frequencies[source_ip] = true_frequencies.get(source_ip, 0) + 1
print(f'true frequency of 35.160.100.86 is: {true_frequencies["35.160.100.86"]}')

true frequency of 35.160.100.86 is: 753


In [43]:
def init_hash_functions(d, w):
    # Generates perfect hash functions from source ips to values between 0 to w - 1.
    hash_functions = []
    for _ in range(d):
        hash_functions.append(dict(map(lambda key, value: (key, value), source_ips.unique(), np.random.randint(0, w, len(source_ips.unique())))))
    return hash_functions

In [44]:
def count_morris_counters(morris_sigma, morris_delta):
    return int(1 / (morris_delta * (morris_sigma ** 2)))

### Count Min Sketch

In [45]:
class CountMinSketch:
    def __init__(self, stream, d, w, morris_sigma=0.2, morris_delta=0.9, use_morris_counters=True):
        self.d = d
        self.w = w
        self.morris_estimators_num = count_morris_counters(morris_sigma, morris_delta)
        self.use_morris_counters = use_morris_counters
        self.hash_functions = init_hash_functions(d, w)
        self.sketch = self.create_sketch(stream)

    def create_sketch(self, stream):
        sketch = {}
        for i, item in enumerate(stream):
            for j, hash_function in enumerate(self.hash_functions):
                if self.use_morris_counters:
                    morris_estimators = sketch.get((j, hash_function[item]), np.zeros(self.morris_estimators_num))
                    p = 1 / (2 ** morris_estimators)
                    update_mask = np.random.uniform(0, 1, size=morris_estimators.shape) <= p
                    morris_estimators[update_mask] += 1
                    sketch[(j, hash_function[item])] = morris_estimators
                else:
                    sketch[(j, hash_function[item])] = sketch.get((j, hash_function[item]), 0) + 1
            sys.stdout.write(f'\rprocessing element {i} of {len(stream)}: {round(float(i / len(stream)) * 100, 2)}% completed.')
        print()
        return sketch

    def query(self, item):
        if self.use_morris_counters:
            hash_values = []
            for i, hash_function in enumerate(self.hash_functions):
                morris_estimators = self.sketch[(i, hash_function[item])]
                processed_morris_estimators = (2 ** morris_estimators) - 1
                hash_values.append(np.mean(processed_morris_estimators))
            return min(hash_values)
        else:
            hash_values = [self.sketch[(i, hash_function[item])] for i, hash_function in enumerate(self.hash_functions)]
            return min(hash_values)

In [46]:
def evaluate_avg_bias(items, sketch, true_frequencies):
    unique_items = items.unique()
    bias_per_item = []
    for item in unique_items:
        true_frequency = true_frequencies[item]
        estimated_frequency = sketch.query(item)
        bias = float(estimated_frequency / true_frequency)
        normalized_bias = 1 - bias if bias <= 1 else bias - 1
        bias_per_item.append(normalized_bias)
    return round(sum(bias_per_item) / len(bias_per_item), 2)

In order to focus the problem we have, we assume that we're given a business requirement to reduce the memory consumption to 10% of the solution that uses true counters.

Since we have 9640 unique elements in the stream and assuming a single counter size is 8 bytes, we need 77,120 bytes to store all the counters. We will try to reduce it to 8KB = 8192 bytes which approximately meets the business requirements.

We will try to optimize the sketch accuracy and overall runtime using only 8 KB.

Using regular counters, we can use up to 2**10 counters:
1. d = 1, w = 1024
2. d = 2, w = 512
3. d = 4, w = 256
4. d = 8, w = 128

In [47]:
if os.path.isfile('count_min_regular_counters_results.pkl'):
    with open('count_min_regular_counters_results.pkl', 'rb') as f:
        results_dict = pickle.load(f)
else:
    results_dict = {}

In [48]:
if not results_dict:
    d_w_pairs = [(1, 1024), (2, 512), (4, 256), (8, 128)]
    elapsed_time = []
    avg_bias = []
    for d, w in d_w_pairs:
        print(f'calculating count min sketch with regular counters with d={d} and w={w}')
        start = time.time()
        count_min_sketch = CountMinSketch(source_ips, d, w, use_morris_counters=False)
        end = time.time()
        elapsed_time.append(int(end - start))
        avg_bias.append(evaluate_avg_bias(source_ips, count_min_sketch, true_frequencies))
    results_dict = {'d_w_pairs': d_w_pairs, 'elapsed_time': elapsed_time, 'avg_bias': avg_bias}
    with open('count_min_regular_counters_results.pkl', 'wb') as f:
        pickle.dump(results_dict, f)

In [49]:
pd.DataFrame.from_dict(results_dict).head()

,elapsed_time,avg_bias,d_w_pairs
0,837,392.04,"(1, 1024)"
1,874,119.05,"(2, 512)"
2,914,194.05,"(4, 256)"
3,921,404.78,"(8, 128)"


We see that the best results are received when d=2, w=512 and d=3, w=256. So for the next experiment we'll try values that are closer to these in order to save time.

In [54]:
print(count_morris_counters(morris_sigma=0.5, morris_delta=0.9))

4


Using morris counters, there's a tradeoff between the amount of matrix counters we use and the amount of morris counters we use to maintain every matrix counter. The storage required for a morris counter is roughly log(64) = 6 bits, therefore a regular counter requires rouhgly the same storage size as 10 morris counters. We don't want to use 10 morris counters per matrix counter because we would prefer using a regular counter and get better accuracy. Our product managers told us that they are willing to tolerate morris_sigma = 0.5 and morris_delta = 0.9, and therefore we can use 4 morris counters per matrix counter. 4 morris counters are 3 bytes, therefore we can use 2730 matrix counters:
 1. d = 2, w = 1365
 2. d = 4, w = 682

Note that for d = 2 we can use w = 1365 matrix counters which is more than twice the amount of regular matrix counters we could use with the same storage capacity.

In [59]:
if os.path.isfile('count_min_morris_counters_results.pkl'):
    with open('count_min_morris_counters_results.pkl', 'rb') as f:
        results_dict = pickle.load(f)
else:
    results_dict = {}

In [60]:
if not results_dict:
    d_w_pairs = [(2, 1365), (4, 682)]
    elapsed_time = []
    avg_bias = []
    morris_sigma = 0.5
    morris_delta = 0.9
    start = time.time()
    for d, w in d_w_pairs:
        print(f'calculating count min sketch with regular counters with d={d} and w={w} and {count_morris_counters(morris_sigma, morris_delta)} morris counters per matrix counter')
        start = time.time()
        count_min_sketch = CountMinSketch(source_ips, d, w, morris_sigma, morris_delta)
        end = time.time()
        elapsed_time.append(int(end - start))
        avg_bias.append(evaluate_avg_bias(source_ips, count_min_sketch, true_frequencies))
    results_dict = {'d_w_pairs': d_w_pairs, 'elapsed_time': elapsed_time, 'avg_bias': avg_bias}
    with open('count_min_morris_counters_results.pkl', 'wb') as f:
        pickle.dump(results_dict, f)

In [61]:
pd.DataFrame.from_dict(results_dict).head()

,d_w_pairs,elapsed_time,avg_bias
0,"(2, 1365)",1269,30.13
1,"(4, 682)",1615,43.45


We see that the best results are received when d=2, w=1365.

The result is 4 times more accurate than the best result we got using regular matrix counters with the same storage capacity, however it also ran 1.5 times slower than with regular matrix counters.

### Count Sketch

In [ ]:
# TODO